In [50]:
# https://nlp.stanford.edu/projects/glove/

In [51]:
from flask import Flask, request, jsonify
from flask_cors import CORS 
# import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
import nltk
import pymysql
import numpy as np
import datetime
app = Flask(__name__)
app.json.sort_keys = False
CORS(app) 

db = pymysql.connect(
    host='mysql5049.site4now.net',
    user='aa0682_movies',
    password='Password1234.',
    db='db_aa0682_movies',
    connect_timeout=8800,
    cursorclass=pymysql.cursors.DictCursor
)
sql_query= """
            SELECT 
                article.article_id, 
                article.title, 
                article.author, 
                article.date, 
                article.date_added,
                article.abstract, 
                journal.journal, 
                article.keyword, 
                files.file_name, 
                COUNT(CASE WHEN logs.type = 'read' THEN 1 END) AS total_reads,
                COUNT(CASE WHEN logs.type = 'download' THEN 1 END) AS total_downloads
            FROM 
                article 
            LEFT JOIN 
                journal ON article.journal_id = journal.journal_id 
            LEFT JOIN 
                logs ON article.article_id = logs.article_id 
            LEFT JOIN 
                files ON article.article_id = files.article_id
            GROUP BY
                article.article_id 
     

           """
db.ping(reconnect=True)
cursor = db.cursor()
cursor.execute(sql_query)
data = cursor.fetchall()


id = [row['article_id'] for row in data]
# overviews_orig = [row['abstract'] for row in data]
overviews = [row['abstract'] for row in data]
titles = [row['title'] for row in data] 
# titles_orig = [row['title']  for row in data] 
# author = [row['author']  for row in data] 
# keyword = [row['keyword']  for row in data] 
# date = [row['date']  for row in data] 


# Preprocessing
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

for n, name in enumerate(overviews):
    temp = name.lower().split(" ")
    temp = [''.join([letter for letter in word if letter.isalnum()]) for word in temp]
    temp = [word for word in temp if word not in stop_words]
    temp = ' '.join(temp)
    overviews[n] = temp
    
for n, title in enumerate(titles):
    temp = title.lower().split(" ")
    temp = [''.join([letter for letter in word if letter.isalnum()]) for word in temp]
    temp = [word for word in temp if word not in stop_words]
    temp = ' '.join(temp)
    titles[n] = temp
    
# Calculate cosine similarity
    from sklearn.feature_extraction.text import CountVectorizer

    vectorizer = CountVectorizer().fit(overviews + titles)
    # Calculate cosine similarity for overviews
    vectorizer_overviews = vectorizer.transform(overviews)
    cosine_sim_overviews = cosine_similarity(vectorizer_overviews)

    # Calculate cosine similarity for titles
    vectorizer_titles =  vectorizer.transform(titles)
    cosine_sim_titles = cosine_similarity(vectorizer_titles)
    
    article_id_to_index = {}  # Create an empty mapping
    for index, article_id in enumerate(id):
        article_id_to_index[article_id] = index
def get_article_recommendations( article_id, overviews_similarity_matrix, titles_similarity_matrix):
    combined_similarity = 0.2 * overviews_similarity_matrix + 0.8 * titles_similarity_matrix
    
    if article_id in article_id_to_index:
        index = article_id_to_index[article_id]
        similar_articles = combined_similarity[index]
        similar_articles = sorted(enumerate(similar_articles), key=lambda x: x[1], reverse=True)
        recommended_articles = []
        
        for i in similar_articles:
            if i[1] < 0.25:
                break
            # recommended_article_title = titles_orig[i[0]]
            # article_description = overviews_orig[i[0]]
            # recommended_articles.append({'title': recommended_article_title, 'article_id': id[i[0]], 'score': i[1]})
            
            recommended_article = {key: data[i[0]][key] for key in data[i[0]]}
            recommended_article['score'] = i[1]
            recommended_articles.append(recommended_article)


        return recommended_articles
    else:
        return ["Article ID not found in the mapping."]

[nltk_data] Downloading package stopwords to C:\Users\My
[nltk_data]     Pc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
import csv
import random
import pickle
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
import matplotlib.pyplot as plt
from scipy.stats import linregress
import random

In [53]:
EMBEDDING_DIM = 100
MAXLEN = 250
TRUNCATING = 'post'
PADDING = 'post'
OOV_TOKEN = "<OOV>"
# MAX_EXAMPLES = 160000
TRAINING_SPLIT = 0.8

In [54]:
# Bundle the two lists into a single one
labels = [ '_'.join(sub['journal'].split(' ')) for n,sub in enumerate(data) ]
sentences_and_labels = list(zip(overviews, labels))

# # Perform random sampling
# random.seed(42)
# sentences_and_labels = random.sample(sentences_and_labels, len(overviews))

# Unpack back into separate lists
sentences, labels = zip(*sentences_and_labels)

print(f"There are {len(sentences)} sentences and {len(labels)} labels\n")

There are 189 sentences and 189 labels



In [79]:
def train_val_split(sentences, labels, training_split, label_encoder):
    """
    Splits the dataset into training and validation sets
    
    Args:
        sentences (list of string): lower-cased sentences without stopwords
        labels (list of string): list of labels
        training split (float): proportion of the dataset to convert to include in the train set
    
    Returns:
        train_sentences, validation_sentences, train_labels, validation_labels - lists containing the data splits
    """    
    ### START CODE HERE
    
    # Compute the number of sentences that will be used for training (should be an integer)
    train_size = int(len(sentences)*training_split)

    ## Shuffle Lists
    temp = list(zip(sentences,labels))
    random.shuffle(temp)
    sentences,labels = zip(*temp)

    # Convert labels strings to integers
    labels = label_encoder.fit_transform(labels)

    # Split the sentences and labels into train/validation splits
    train_sentences = sentences[:train_size]
    train_labels = labels[:train_size]

    validation_sentences = sentences[train_size:]
    validation_labels = labels[train_size:]
    
    ### END CODE HERE
    
    return train_sentences, validation_sentences, train_labels, validation_labels, label_encoder

In [80]:
# Test your function
label_encoder = LabelEncoder()
train_sentences, val_sentences, train_labels, val_labels, label_encoder = train_val_split(sentences, labels, TRAINING_SPLIT, label_encoder)

print(f"There are {len(train_sentences)} sentences for training.\n")
print(f"There are {len(train_labels)} labels for training.\n")
print(f"There are {len(val_sentences)} sentences for validation.\n")
print(f"There are {len(val_labels)} labels for validation.")

There are 151 sentences for training.

There are 151 labels for training.

There are 38 sentences for validation.

There are 38 labels for validation.


In [57]:
def fit_tokenizer(train_sentences, oov_token):
    """
    Instantiates the Tokenizer class on the training sentences
    
    Args:
        train_sentences (list of string): lower-cased sentences without stopwords to be used for training
        oov_token (string) - symbol for the out-of-vocabulary token
    
    Returns:
        tokenizer (object): an instance of the Tokenizer class containing the word-index dictionary
    """
    ### START CODE HERE
    
    # Instantiate the Tokenizer class, passing in the correct value for oov_token
    tokenizer = Tokenizer(oov_token=oov_token)
    
    # Fit the tokenizer to the training sentences
    tokenizer.fit_on_texts(train_sentences)
    
    ### END CODE HERE
    
    return tokenizer

In [58]:
# Test your function
tokenizer = fit_tokenizer(train_sentences, OOV_TOKEN)

word_index = tokenizer.word_index
VOCAB_SIZE = len(word_index)

print(f"Vocabulary contains {VOCAB_SIZE} words\n")
print("<OOV> token included in vocabulary" if "<OOV>" in word_index else "<OOV> token NOT included in vocabulary")
print(f"\nindex of word 'gender' should be {word_index['gender']}")

Vocabulary contains 6283 words

<OOV> token included in vocabulary

index of word 'gender' should be 304


In [59]:
def seq_pad_and_trunc(sentences, tokenizer, padding, truncating, maxlen):
    """
    Generates an array of token sequences and pads them to the same length
    
    Args:
        sentences (list of string): list of sentences to tokenize and pad
        tokenizer (object): Tokenizer instance containing the word-index dictionary
        padding (string): type of padding to use
        truncating (string): type of truncating to use
        maxlen (int): maximum length of the token sequence
    
    Returns:
        pad_trunc_sequences (array of int): tokenized sentences padded to the same length
    """        
    ### START CODE HERE
       
    # Convert sentences to sequences
    sequences = tokenizer.texts_to_sequences(sentences)
    
    # Pad the sequences using the correct padding, truncating and maxlen
    pad_trunc_sequences = pad_sequences(sequences, maxlen=maxlen, padding=padding, truncating=truncating)
    
    ### END CODE HERE
    
    return pad_trunc_sequences

In [60]:
# Test your function
train_pad_trunc_seq = seq_pad_and_trunc(train_sentences, tokenizer, PADDING, TRUNCATING, MAXLEN)
val_pad_trunc_seq = seq_pad_and_trunc(val_sentences, tokenizer, PADDING, TRUNCATING, MAXLEN)

print(f"Padded and truncated training sequences have shape: {train_pad_trunc_seq.shape}\n")
print(f"Padded and truncated validation sequences have shape: {val_pad_trunc_seq.shape}")

Padded and truncated training sequences have shape: (151, 250)

Padded and truncated validation sequences have shape: (38, 250)


In [61]:
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)

In [62]:
# Define path to file containing the embeddings
GLOVE_FILE = './glove.6B.100d.txt'

# Initialize an empty embeddings index dictionary
GLOVE_EMBEDDINGS = {}

# Read file and fill GLOVE_EMBEDDINGS with its contents
with open(GLOVE_FILE, encoding="utf8", errors="ignore") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        GLOVE_EMBEDDINGS[word] = coefs

In [63]:
test_word = 'gender'

test_vector = GLOVE_EMBEDDINGS[test_word]

print(f"Vector representation of word {test_word} looks like this:\n\n{test_vector}")

Vector representation of word gender looks like this:

[ 0.7528     0.58694    0.10059    0.45384   -0.59904    0.3756
 -0.48809    0.3255     0.66021    1.0741    -0.87795   -0.070058
  0.38828    0.17249    1.198     -0.14775    0.48218    0.0080089
 -0.072508  -0.29931    0.050217  -0.7677     0.40769    0.52481
 -0.33323   -0.70466    0.48616   -1.4025    -0.32132   -0.11926
 -0.28853    0.29444    0.12296   -0.27784    0.64675   -0.80821
  0.24569    0.91006   -0.69788    0.78162   -0.75286   -0.309
 -0.74415    0.13453   -1.1376    -0.21934   -0.043231   0.90301
 -1.6366    -0.77089    0.37707   -0.070822  -1.0185     1.0259
  0.28272   -1.0935     0.46933   -0.21428    0.63936    0.1496
 -0.12656   -0.19598   -0.10937   -0.50143    1.2136    -0.046943
  0.57951   -0.55169    0.25192    0.62123   -0.12948    0.58623
 -0.19002    0.056338   0.40258   -0.11338   -0.61292   -0.74152
 -0.49465   -0.98182   -1.1541    -0.39493    0.82455    0.1865
 -1.6896    -0.035706  -0.88967   -0.

In [64]:
# Initialize an empty numpy array with the appropriate size
EMBEDDINGS_MATRIX = np.zeros((VOCAB_SIZE+1, EMBEDDING_DIM))

# Iterate all of the words in the vocabulary and if the vector representation for 
# each word exists within GloVe's representations, save it in the EMBEDDINGS_MATRIX array
for word, i in word_index.items():
    embedding_vector = GLOVE_EMBEDDINGS.get(word)
    if embedding_vector is not None:
        EMBEDDINGS_MATRIX[i] = embedding_vector

In [68]:
# GRADED FUNCTION: create_model
def create_model(vocab_size, embedding_dim, maxlen, embeddings_matrix):
    """
    Creates a binary sentiment classifier model
    
    Args:
        vocab_size (int): size of the vocabulary for the Embedding layer input
        embedding_dim (int): dimensionality of the Embedding layer output
        maxlen (int): length of the input sequences
        embeddings_matrix (array): predefined weights of the embeddings
    
    Returns:
        model (tf.keras Model): the sentiment classifier model
    """
    ### START CODE HERE
    
    model = tf.keras.Sequential([ 
        # This is how you need to set the Embedding layer when using pre-trained embeddings
        tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=maxlen, weights=[embeddings_matrix], trainable=False), 
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, activation="relu", return_sequences=True)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, activation="relu")),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')
    ])
    
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=optimizers.Adam(1e-4),
                  metrics=['accuracy']) 

    ### END CODE HERE

    return model

In [69]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=3, min_lr=1e-7)
es = EarlyStopping(monitor='val_loss', patience=5, mode="min", restore_best_weights=True)
def scheduler(epoch, lr):
  if epoch < 8:
    return lr
  else:
    return lr * tf.math.exp(-0.1)
schedule = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [70]:
# Create your untrained model
model = create_model(VOCAB_SIZE, EMBEDDING_DIM, MAXLEN, EMBEDDINGS_MATRIX)

# Train the model and save the training history
history = model.fit(train_pad_trunc_seq, train_labels, epochs=32, validation_data=(val_pad_trunc_seq, val_labels), callbacks=[reduce_lr, es, schedule])

Epoch 1/32
5/5 [==============================] - 12s 2s/step - loss: 1.0894 - accuracy: 0.4570 - val_loss: 1.0717 - val_accuracy: 0.7105 - lr: 1.0000e-04
Epoch 2/32
5/5 [==============================] - 8s 2s/step - loss: 1.0509 - accuracy: 0.5629 - val_loss: 1.0423 - val_accuracy: 0.6579 - lr: 1.0000e-04
Epoch 3/32
5/5 [==============================] - 9s 2s/step - loss: 1.0167 - accuracy: 0.5563 - val_loss: 1.0131 - val_accuracy: 0.6316 - lr: 1.0000e-04
Epoch 4/32
5/5 [==============================] - 9s 2s/step - loss: 0.9821 - accuracy: 0.5497 - val_loss: 0.9782 - val_accuracy: 0.6316 - lr: 1.0000e-04
Epoch 5/32
5/5 [==============================] - 8s 2s/step - loss: 0.9417 - accuracy: 0.5563 - val_loss: 0.9383 - val_accuracy: 0.6316 - lr: 1.0000e-04
Epoch 6/32
5/5 [==============================] - 8s 2s/step - loss: 0.9000 - accuracy: 0.5430 - val_loss: 0.8913 - val_accuracy: 0.6316 - lr: 1.0000e-04
Epoch 7/32
5/5 [==============================] - 8s 2s/step - loss: 0.8539

In [87]:
## Input Preprocessing
label_encoder.inverse_transform([np.argmax(())])

5/5 [==============================] - 2s 310ms/step


array(['The_Gavel'], dtype='<U9')

In [133]:
number = 100
test_data = train_pad_trunc_seq[number].reshape(1,250)
ground_truth = train_labels[number]
classification = model.predict(test_data)

print('Ground Truth:', label_encoder.inverse_transform([(ground_truth)]))
print('Classification Truth:', label_encoder.inverse_transform([np.argmax((classification))]))

print('Actual Input Sentence', )


1/1 [==============================] - 0s 292ms/step
Ground Truth: ['The_Gavel']
Classification Truth: ['The_Gavel']
Actual Input Sentence
